# Integrated Bioinformatics Project : Flux_Balance_F_Prausnitzii 
#### Students: Michael Shawn Neilsson, Biancamaria FLorenzi, Prabhat Juyal, Tim Blokker
#### Supervisors: Clémence Joseph, Karoline Faust

In [163]:
#import packages
from libchebipy import ChebiEntity
import json
import requests
import molmass
import io
import cobra
import functions
import re
from functions import *
import cobrababel
import pandas as pd

In [164]:
#model = cobra.io.read_sbml_model('Data/f_prausnitzii.xml')
#info(model)
model=cobra.io.load_matlab_model('Data/iFpraus_v_1_0.mat')
info(model)


Metabolites :  833
Reactions :  1030
Genes :  602
Compartments :  {'c': '', 'e': ''}
Objective function : 
 Maximize
1.0*Biomass_FP - 1.0*Biomass_FP_reverse_ee33b 



In [165]:
x=[model.reactions.get_by_id(compound_id).name for compound_id in model.medium]
x=[compound[:(len(compound)-3)] for compound in x]
x=[compoundname.replace("EX_", "") for compoundname in x]
x=[compoundname.replace("excha", "").strip() for compoundname in x]
#";".join(x)

In [166]:
mapped_compounds=Mol_Weight(x)
mapped_compounds["BiGG_ID"]=[model.reactions.get_by_id(compound_id).id[3:] for compound_id in model.medium]
weights=mapped_compounds[["query", "BiGG_ID", "hit", "mol_weight"]]

In [167]:
#outputmodel= cobra.io.save_json_model(model, "Data/iFpraus.json")

In [168]:
#unaltered model
model1=model.copy()
print("\nModel Medium")
model1.objective = {model1.reactions.get_by_id('Biomass_FP'): 1}
print(model1.summary())


Model Medium
Objective
1.0 Biomass_FP = 69.87885590787737

Uptake
------
    Metabolite          Reaction   Flux  C-Number  C-Flux
    Lcystin[e]     EX_Lcystin(e)  3.502         6   0.05%
        ade[e]         EX_ade(e)  864.9         5  10.85%
     adocbl[e]      EX_adocbl(e) 0.3929        72   0.07%
     alaala[e]      EX_alaala(e)  6.462         6   0.10%
     alagln[e]      EX_alagln(e)  30.81         8   0.62%
      arg-L[e]       EX_arg_L(e)   1000         6  15.05%
      asn-L[e]       EX_asn_L(e)   16.4         4   0.16%
        btn[e]         EX_btn(e) 0.3929        10   0.01%
        ca2[e]         EX_ca2(e) 0.3929         0   0.00%
         cl[e]          EX_cl(e) 0.3929         0   0.00%
    cobalt2[e]     EX_cobalt2(e) 0.3929         0   0.00%
     cps_fp[e]      EX_cps_fp(e) 0.3929        63   0.06%
        cu2[e]         EX_cu2(e) 0.3929         0   0.00%
        fe3[e]         EX_fe3(e) 0.7858         0   0.00%
        fol[e]         EX_fol(e)  1.179        19   0.06

### Gifu Anaerobic Medium (mGAM) from:
https://hyserve.com/files/05433_GAM-Broth_Modified_final.pdf


- Peptone, Soya Peptone, Proteose Peptone: source of amino acids -> most likely all of them 
- Yeast extract, liver extract, meat extract, digested serum -> 
    - amino acids, 
    - '4abz(e)' 
    - '5oxpro(e)'
    - thm(e)' thiamin vitamin b1
    - biotin -> btn |from yeast extract? 
    - 'rbflvrd(e)', 'ribflv(e)' vitamin b2
    - 'pydam(e)' pydx pydxn vitamin b6 from yeast extract
    - 'pnto_R(e)' pantothenic acid from yeast extract?
    - 'ptrc(e)' probably from meat extract(e)
    - 'spmd(e)' from meat extract and linked to ptrc
    - cbl1   Cyanocobalamin is a cobalt-containing coordination compound and a natural water-soluble vitamin of the B-complex family 
    - adocbl
    - folate fol is of B vitamin family 
    - tautin from meat extract
    - nac is nicotinate and also vitamin b and so is nacm nicotinamide
    - hxan, gua , ins are all some kind of nucleic acid that is probably present in yeast extract
    - trace elements: Ca2? Cu2? cobalt2?  mg2? pi(e)? so4?
- Dextrose -> same as glucose (https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:17634) and so -> glc_D
- Soluble starch -> starch1 (potatoe starch)
- L-Tryptophane -> trp__L
- L-Cystein Hydrochloride -> cys_L and cl and h (hydrochloride<-> HCL becomes cl and h)
- Sodium Thioglycolate -> thiog and na1 but it is just a marker for tolerance of oxygen apparently so ignore
- L-Arginine -> arg_L
- Vitamine K1 -> phllqne https://pubchem.ncbi.nlm.nih.gov/compound/Vitamin-K1#section=Depositor-Supplied-Synonyms 
- Hemin -> Iron and Chlorine  fe3 and cl  and perhaps ppp9 https://pubchem.ncbi.nlm.nih.gov/#query=3-%5B18-(2-carboxyethyl)-8%2C13-bis(ethenyl)-3%2C7%2C12%2C17-tetramethylporphyrin-21%2C23-diid-2-yl%5Dpropanoic%20acid%3Biron(3%2B)%3Bchloride 
- Potassium Dihydrogen Phosphate -> pi, k, h
- Sodium Chloride -> na1 cl

Digested serum : https://www.nebiolabs.com.au/-/media/catalog/datacards-or-manuals/p8108datasheet-lot0021306.pdf	

Generally useful page ; https://pubchem.ncbi.nlm.nih.gov/#query=Hemin

check also: https://opencobra.github.io/cobratoolbox/latest/tutorials/tutorialMetabotoolsI.html 

In [169]:
import numpy as np
aa_pre=("ala,arg,asn,asp,asx,cys,glu,gln,glx,gly,his,ile,leu,lys,met,phe,pro,ser,thr,trp,tyr,val").split(",")
aa=[]
for aminoacid in aa_pre:
    if aminoacid != "gly": 
        aa.append(aminoacid +"_L(e)")
    else:
        aa.append(aminoacid+'(e)')
aa=sorted(aa)
aa=np.array(aa)
aa

array(['ala_L(e)', 'arg_L(e)', 'asn_L(e)', 'asp_L(e)', 'asx_L(e)',
       'cys_L(e)', 'gln_L(e)', 'glu_L(e)', 'glx_L(e)', 'gly(e)',
       'his_L(e)', 'ile_L(e)', 'leu_L(e)', 'lys_L(e)', 'met_L(e)',
       'phe_L(e)', 'pro_L(e)', 'ser_L(e)', 'thr_L(e)', 'trp_L(e)',
       'tyr_L(e)', 'val_L(e)'], dtype='<U8')

In [170]:
MediumMgam_complete=[x[3:len(x)] for x in model.medium]
print(MediumMgam_complete)

['3mop(e)', '4abz(e)', '5oxpro(e)', 'Lcyst(e)', 'Lcystin(e)', 'ac(e)', 'acasp(e)', 'acgal(e)', 'acgalglcur(e)', 'acgam(e)', 'acglu(e)', 'acnam(e)', 'ade(e)', 'adocbl(e)', 'ala_D(e)', 'ala_L(e)', 'alaala(e)', 'alaasp(e)', 'alagln(e)', 'alaglu(e)', 'alagly(e)', 'alahis(e)', 'alaleu(e)', 'alathr(e)', 'arab_L(e)', 'arg_L(e)', 'asn_L(e)', 'aso3(e)', 'aso4(e)', 'asp_L(e)', 'btn(e)', 'but(e)', 'butso3(e)', 'ca2(e)', 'cbl1(e)', 'cbl2(e)', 'cd2(e)', 'cellb(e)', 'cgly(e)', 'cit(e)', 'cl(e)', 'co2(e)', 'cobalt2(e)', 'cps_fp(e)', 'cu2(e)', 'cys_L(e)', 'dhor_S(e)', 'ethso3(e)', 'fe2(e)', 'fe3(e)', 'fe3dcit(e)', 'fol(e)', 'for(e)', 'fru(e)', 'fum(e)', 'gal(e)', 'galur(e)', 'gam(e)', 'glc(e)', 'glcur(e)', 'gln_L(e)', 'glu_L(e)', 'gly(e)', 'glyasn(e)', 'glyasp(e)', 'glyc3p(e)', 'glyc_R(e)', 'glygln(e)', 'glyglu(e)', 'glygly(e)', 'glyleu(e)', 'glymet(e)', 'glyphe(e)', 'glypro(e)', 'glytyr(e)', 'gthox(e)', 'gthrd(e)', 'gua(e)', 'h(e)', 'h2o(e)', 'h2s(e)', 'hg2(e)', 'his_L(e)', 'hxan(e)', 'ile_L(e)', 'in

In [171]:
MediumMgam_complete=[x[3:len(x)] for x in model.medium]

Di_AA=['acglu(e)', 'acasp(e)','alaala(e)', 'alaasp(e)', 'alagln(e)', 'alaglu(e)', 'alagly(e)', 'alahis(e)', 'alaleu(e)', 'alathr(e)',     'glyasn(e)', 'glyasp(e)',  'glygln(e)', 'glyglu(e)', 'glygly(e)', 'glyleu(e)', 'glymet(e)', 'glyphe(e)', 'glypro(e)', 'glytyr(e)','leugly(e)', 'leuleu(e)',   'metala(e)', 'metsox_R_L(e)', 'metsox_S_L(e)', 'progly(e)']

D_AA=['ala_D(e)','met_D(e)']

Not_in_Peptones_Oxoid=['asn_L(e)', 'asx_L(e)','gln_L(e)','glx_L(e)', 'his_L(e)','3mop(e)', '5oxpro(e)']

Carbohydrates_not_in_Medium=['ac(e)',  'acgal(e)', 'acgalglcur(e)', 'acgam(e)','arab_L(e)','but(e)', 'butso3(e)', 'co2(e)', 'fru(e)', 'fum(e)', 'gal(e)', 'galur(e)', 'gam(e)', 'glcur(e)', 'glyc3p(e)', 'ethso3(e)', 'for(e)',  'kesto(e)', 'kestopt(e)', 'kestottr(e)', 'lac_D(e)', 'lcts(e)', 'pect(e)', 'plac(e)', 'succ(e)', 'mal_L(e)', 'malt(e)', 'man(e)', 'meoh(e)']

not_classified_but_removed=['4abz(e)',  'acnam(e)', 'aso3(e)', 'aso4(e)', 'cd2(e)', 'cellb(e)', 'cgly(e)', 'cit(e)',  'cps_fp(e)', 'dhor_S(e)',  'fe3dcit(e)', 'glyc_R(e)', 'h2s(e)', 'hg2(e)', 'inulin(e)', 'isetac(e)',  'met_D(e)', 'mobd(e)', 'mso3(e)', 'nh4(e)', 'o2(e)', 'orn(e)', 'orot(e)',  'seln(e)',  'sulfac(e)', 'urate(e)', 'urea(e)']

MediumMgam =[comp for comp in MediumMgam_complete if comp not in (Di_AA + D_AA + Not_in_Peptones_Oxoid + Carbohydrates_not_in_Medium + not_classified_but_removed)]
print(MediumMgam, '\nLength MediumMgam: ', len(MediumMgam))

['Lcyst(e)', 'Lcystin(e)', 'ade(e)', 'adocbl(e)', 'ala_L(e)', 'arg_L(e)', 'asp_L(e)', 'btn(e)', 'ca2(e)', 'cbl1(e)', 'cbl2(e)', 'cl(e)', 'cobalt2(e)', 'cu2(e)', 'cys_L(e)', 'fe2(e)', 'fe3(e)', 'fol(e)', 'glc(e)', 'glu_L(e)', 'gly(e)', 'gthox(e)', 'gthrd(e)', 'gua(e)', 'h(e)', 'h2o(e)', 'hxan(e)', 'ile_L(e)', 'ins(e)', 'k(e)', 'leu_L(e)', 'lys_L(e)', 'met_L(e)', 'mg2(e)', 'na1(e)', 'nac(e)', 'ncam(e)', 'pb(e)', 'phe_L(e)', 'pi(e)', 'pnto_R(e)', 'pro_L(e)', 'ptrc(e)', 'pydam(e)', 'pydx(e)', 'pydxn(e)', 'rbflvrd(e)', 'ribflv(e)', 'ser_L(e)', 'so4(e)', 'spmd(e)', 'strch1(e)', 'taur(e)', 'thm(e)', 'thr_L(e)', 'thymd(e)', 'trp_L(e)', 'tyr_L(e)', 'ura(e)', 'val_L(e)', 'xan(e)', 'xyl_D(e)'] 
Length MediumMgam:  62


In [172]:
not_kept=[comp for comp in MediumMgam_complete if comp not in MediumMgam]
kept=[comp for comp in MediumMgam_complete if comp in MediumMgam]
print('Length MediumMgam_complete: ', len(MediumMgam_complete),'\n\n', not_kept,'\nLenght Not_kept: ' ,len(not_kept), '\n\n', kept, '\nLength Kept: ', len(kept))

Length MediumMgam_complete:  150 

 ['3mop(e)', '4abz(e)', '5oxpro(e)', 'ac(e)', 'acasp(e)', 'acgal(e)', 'acgalglcur(e)', 'acgam(e)', 'acglu(e)', 'acnam(e)', 'ala_D(e)', 'alaala(e)', 'alaasp(e)', 'alagln(e)', 'alaglu(e)', 'alagly(e)', 'alahis(e)', 'alaleu(e)', 'alathr(e)', 'arab_L(e)', 'asn_L(e)', 'aso3(e)', 'aso4(e)', 'but(e)', 'butso3(e)', 'cd2(e)', 'cellb(e)', 'cgly(e)', 'cit(e)', 'co2(e)', 'cps_fp(e)', 'dhor_S(e)', 'ethso3(e)', 'fe3dcit(e)', 'for(e)', 'fru(e)', 'fum(e)', 'gal(e)', 'galur(e)', 'gam(e)', 'glcur(e)', 'gln_L(e)', 'glyasn(e)', 'glyasp(e)', 'glyc3p(e)', 'glyc_R(e)', 'glygln(e)', 'glyglu(e)', 'glygly(e)', 'glyleu(e)', 'glymet(e)', 'glyphe(e)', 'glypro(e)', 'glytyr(e)', 'h2s(e)', 'hg2(e)', 'his_L(e)', 'inulin(e)', 'isetac(e)', 'kesto(e)', 'kestopt(e)', 'kestottr(e)', 'lac_D(e)', 'lcts(e)', 'leugly(e)', 'leuleu(e)', 'mal_L(e)', 'malt(e)', 'man(e)', 'meoh(e)', 'met_D(e)', 'metala(e)', 'metsox_R_L(e)', 'metsox_S_L(e)', 'mobd(e)', 'mso3(e)', 'nh4(e)', 'o2(e)', 'orn(e)', 'orot(

In [173]:
#µg_L =mg/L =0.001 g/L
#http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=LP0037&c=UK&lang=EN&minfo=Y
Traces_BiGG_ID=["ca2(e)" , "cl(e)", "cu2(e)", "fe2(e)", "fe3(e)", "k(e)",  "mg2(e)", "na1(e)","pb(e)"]

Bact_Peptone_L37=5#g/L
Proteose_Peptone_L85=5
soya_peptone_L44=3
yeast_extract=2.5
Lab_Lemco_L29=2.2
Liver_Digest_Neutralized_L27=1.2

conc_µg_L_Bact_Peptone_L37=              np.array([635, 1*10*1000, 1, 22/2, 22/2, 3.6*10*1000, 265, 1*10*1000, 0.4])*Bact_Peptone_L37#10^-6*g/L 0= µg/L
conc_µg_L_Proteose_Peptone_L85=          np.array( [200, 8*10*1000, 10, 42/2, 42/2, 1.4*10*1000, 340, 8*10*1000, 0.1])*Proteose_Peptone_L85
conc_µg_L_soya_peptone_L44=              np.array( [225,0.4*10*1000, 2,90/2,90/2, 3.3*10*1000, 1530, 0.4*10*1000, 0.3])*soya_peptone_L44
conc_µg_L_yeast_extract=                 np.array( [155,0.3*10*1000, 2,52/2,52/2, 7.3*10*1000, 205, 0.3*10*1000, 0.7])*yeast_extract
conc_µg_L_Lab_Lemco_L29=                 np.array( [140,1.1*10*1000, 2,20/2,20/2, 1.9*10*1000, 140, 1.1*10*1000, 0.3])*Lab_Lemco_L29#meat extract
conc_µg_L_Liver_Digest_Neutralized_L27=  np.array([98,2.2*10*1000, 12,245/2,245/2, 1.6*10*1000, 210, 2.2*10*1000, 0.2])*Liver_Digest_Neutralized_L27 #liver extract

Traces=pd.DataFrame({"BiGG_ID": Traces_BiGG_ID, 
                    "conc_µg_L_Bact_Peptone_L37":conc_µg_L_Bact_Peptone_L37, 
                    "conc_µg_L_Proteose_Peptone_L85":conc_µg_L_Proteose_Peptone_L85,
                    "conc_µg_L_soya_peptone_L44":conc_µg_L_soya_peptone_L44, 
                    "conc_µg_L_yeast_extract":conc_µg_L_yeast_extract, 
                    'conc_µg_L_Lab_Lemco_L29':conc_µg_L_Lab_Lemco_L29,
                    'conc_µg_L_Liver_Digest_Neutralized_L27' :conc_µg_L_Liver_Digest_Neutralized_L27 })

In [174]:
#AA from peptones and yeast extract
#http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=LP0037&c=UK&lang=EN&minfo=Y

percent_to_ppm=10*1000

index_notinPeptones= np.where(np.isin(aa, ['asn_L(e)', 'asx_L(e)','gln_L(e)','glx_L(e)', 'his_L(e)']))
index_notinPeptones=sorted(index_notinPeptones[0], reverse=True)
aa_BiGG_ID=np.delete(aa, index_notinPeptones)
aa_conc_µg_L_Bact_Peptone_L37=np.array([3.92,4.99, 6.06, 1.66, 9.93, 7.71, 3.81, 3.79, 4.38, 1.58, 2.60, 5.83, 2.81, 1.25, 0.66, 0.39, 3.33])*percent_to_ppm*Bact_Peptone_L37
aa_conc_µg_L_Proteose_Peptone_L85=np.array([3.81, 5.80, 5.85, 0.15, 13.78, 4.45, 4.58, 6.01, 4.61, 1.08, 4.66, 5.99, 2.18, 2.75, 0.75, 1.77, 4.11])*percent_to_ppm*Proteose_Peptone_L85
aa_conc_µg_L_soya_peptone_L44=np.array([2.57, 4.64, 7.06, 0.53, 14.71, 2.83, 2.51, 4.31, 3.77, 0.62, 0.38, 3.40, 0.67, 1.68, 0.64, 2.09, 3.65])*percent_to_ppm*soya_peptone_L44	
aa_conc_Yeast_Extract_L21=np.array([0.91, 3.31, 7.07, 0.76, 13.49, 5.95, 4.81, 6.04, 5.40, 0.80, 3.78, 0.88, 3.42, 2.73, 0.85, 4.95, 1.00])*percent_to_ppm*yeast_extract
aa_conc_µg_L_Lab_Lemco_L29=np.array([5.85, 7.10, 5.10, 0.68, 10.71, 10.85, 3.17, 3.15, 4.78, 2.61, 2.34, 7.79, 1.87, 2.54, 0.34, 0.66, 3.06 ])*percent_to_ppm*Lab_Lemco_L29#meat extract
aa_conc_µg_L_Liver_Digest_Neutralized_L27=np.array([3.40, 2.89, 5.85, 0.34, 10.38, 3.83, 2.11, 5.96, 6.02, 0.95, 2.45, 2.38, 0.83, 1.42, 1.02, 2.54, 3.05 ])*percent_to_ppm*Liver_Digest_Neutralized_L27 #liver extract

AA=pd.DataFrame({"BiGG_ID": aa_BiGG_ID, 
                    "conc_µg_L_Bact_Peptone_L37":aa_conc_µg_L_Bact_Peptone_L37,
                    "conc_µg_L_Proteose_Peptone_L85":aa_conc_µg_L_Proteose_Peptone_L85,
                    "conc_µg_L_soya_peptone_L44":aa_conc_µg_L_soya_peptone_L44,
                    "conc_µg_L_yeast_extract":aa_conc_Yeast_Extract_L21,  
                    'conc_µg_L_Lab_Lemco_L29':aa_conc_µg_L_Lab_Lemco_L29,
                    'conc_µg_L_Liver_Digest_Neutralized_L27' : aa_conc_µg_L_Liver_Digest_Neutralized_L27 })


In [175]:
Weights_Mgam=weights[weights["BiGG_ID"].isin(MediumMgam)]
Traces_AA_µg_L=AA.append(Traces)
Traces_AA_µg_L=pd.merge(Traces_AA_µg_L, Weights_Mgam, on='BiGG_ID')
Traces_AA_µg_L

,BiGG_ID,conc_µg_L_Bact_Peptone_L37,conc_µg_L_Proteose_Peptone_L85,conc_µg_L_soya_peptone_L44,conc_µg_L_yeast_extract,conc_µg_L_Lab_Lemco_L29,conc_µg_L_Liver_Digest_Neutralized_L27,query,hit,mol_weight
0,ala_L(e),196000.0,190500.0,77100.0,22750.00,128700.00,40800.00,L-Alanine,L-Alanine,89.09322
1,arg_L(e),249500.0,290000.0,139200.0,82750.00,156200.00,34680.00,L-Arginine,L-Arginine,174.20100
2,asp_L(e),303000.0,292500.0,211800.0,176750.00,112200.00,70200.00,L-Aspartate,L-Aspartic acid,133.10270
3,cys_L(e),83000.0,7500.0,15900.0,19000.00,14960.00,4080.00,L-Cysteine,L-Cysteine,121.15800
4,glu_L(e),496500.0,689000.0,441300.0,337250.00,235620.00,124560.00,L-Glutamate,L-Glutamic acid,147.12930
5,gly(e),385500.0,222500.0,84900.0,148750.00,238700.00,45960.00,Glycine,Glycine,75.06664
6,ile_L(e),190500.0,229000.0,75300.0,120250.00,69740.00,25320.00,L-Isoleucine,L-Isoleucine,131.17500
7,leu_L(e),189500.0,300500.0,129300.0,151000.00,69300.00,71520.00,L-Leucine,L-Leucine,131.17296
8,lys_L(e),219000.0,230500.0,113100.0,135000.00,105160.00,72240.00,L-Lysine,L-Lysine,146.18764
9,met_L(e),79000.0,54000.0,18600.0,20000.00,57420.00,11400.00,L-Methionine,L-Methionine,149.21238


In [176]:
col_list= list(Traces_AA_µg_L)
col_list.remove('mol_weight')

In [177]:
µg_to_mg=1/1000
if "sum_µg_L" not in Traces_AA_µg_L.columns:
    Traces_AA_µg_L["sum_µg_L"]=Traces_AA_µg_L[col_list].sum(numeric_only=True, axis=1)
Traces_AA_µg_L["mg_L"]=np.array(Traces_AA_µg_L["sum_µg_L"])*µg_to_mg
Traces_AA_µg_L

,BiGG_ID,conc_µg_L_Bact_Peptone_L37,conc_µg_L_Proteose_Peptone_L85,conc_µg_L_soya_peptone_L44,conc_µg_L_yeast_extract,conc_µg_L_Lab_Lemco_L29,conc_µg_L_Liver_Digest_Neutralized_L27,query,hit,mol_weight,sum_µg_L,mg_L
0,ala_L(e),196000.0,190500.0,77100.0,22750.00,128700.00,40800.00,L-Alanine,L-Alanine,89.09322,655850.00,655.85000
1,arg_L(e),249500.0,290000.0,139200.0,82750.00,156200.00,34680.00,L-Arginine,L-Arginine,174.20100,952330.00,952.33000
2,asp_L(e),303000.0,292500.0,211800.0,176750.00,112200.00,70200.00,L-Aspartate,L-Aspartic acid,133.10270,1166450.00,1166.45000
3,cys_L(e),83000.0,7500.0,15900.0,19000.00,14960.00,4080.00,L-Cysteine,L-Cysteine,121.15800,144440.00,144.44000
4,glu_L(e),496500.0,689000.0,441300.0,337250.00,235620.00,124560.00,L-Glutamate,L-Glutamic acid,147.12930,2324230.00,2324.23000
5,gly(e),385500.0,222500.0,84900.0,148750.00,238700.00,45960.00,Glycine,Glycine,75.06664,1126310.00,1126.31000
6,ile_L(e),190500.0,229000.0,75300.0,120250.00,69740.00,25320.00,L-Isoleucine,L-Isoleucine,131.17500,710110.00,710.11000
7,leu_L(e),189500.0,300500.0,129300.0,151000.00,69300.00,71520.00,L-Leucine,L-Leucine,131.17296,911120.00,911.12000
8,lys_L(e),219000.0,230500.0,113100.0,135000.00,105160.00,72240.00,L-Lysine,L-Lysine,146.18764,875000.00,875.00000
9,met_L(e),79000.0,54000.0,18600.0,20000.00,57420.00,11400.00,L-Methionine,L-Methionine,149.21238,240420.00,240.42000


In [178]:
#Defined ingredients
g_to_mg=1000
defined=np.array(['glc(e)','strch1(e)', 'trp_L(e)', 'cys_L(e)', 'h(e)', 'cl(e)', 'na1(e)', 'arg_L(e)', 'k(e)', 'pi(e)'])
conc_defined_g_l=np.array([0.5, 5, 0.2, 0.3, 0.3, 0.3+3, 0.3+3, 1, 2.5, 2.5])*g_to_mg
defined_g_L=pd.merge(pd.DataFrame({'BiGG_ID':defined, 'mg_L':conc_defined_g_l}), Weights_Mgam, on='BiGG_ID')
defined_g_L

,BiGG_ID,mg_L,query,hit,mol_weight
0,glc(e),500.0,D-Glucose,D-Glucose,180.15588
1,strch1(e),5000.0,"starch, structure 1 (1,6-{7[1,4-Glc], 4[1,4-Gl...",,NaN
2,trp_L(e),200.0,L-Tryptophan,L-Tryptophan,204.22526
3,cys_L(e),300.0,L-Cysteine,L-Cysteine,121.15800
4,h(e),300.0,H,L-Histidine,155.15468
5,cl(e),3300.0,Chloride,Chloride ion,35.45270
6,na1(e),3300.0,Sodium,Sodium,22.98977
7,arg_L(e),1000.0,L-Arginine,L-Arginine,174.20100
8,k(e),2500.0,K,L-Lysine,146.18764
9,pi(e),2500.0,Phosphate,Phosphate,97.99520


In [179]:
Traces_AA_def=Traces_AA_µg_L[["BiGG_ID", "query", "hit", "mg_L", "mol_weight"]].append(defined_g_L).reset_index()

missing_weights_BiGG_ID=np.array(Traces_AA_def[Traces_AA_def['mol_weight'].isnull()]["BiGG_ID"])
missing_weights=Mol_Weight(["Copper", "Fe", "Magnesium", "Maltoundecadose"]) #Maltoundecadose equals 
Fe=missing_weights.loc[1:1]
missing_weights_weights = pd.concat([missing_weights.loc[:1], Fe, missing_weights.loc[2:]]).reset_index(drop=True)
missing_weights_weights.loc[4:4]["mol_weight"]=1801.6 #g/mol https://pubchem.ncbi.nlm.nih.gov/compound/Maltoundecaose#section=Computed-Properties 

In [180]:
index=Traces_AA_def[Traces_AA_def["mol_weight"].isnull()].index
for i in range(0,len(index)):
    Traces_AA_def.loc[index[i],"mol_weight"]= missing_weights_weights["mol_weight"][i]
Traces_AA_def["mmol/L"]=Traces_AA_def["mg_L"]/Traces_AA_def["mol_weight"]
Traces_AA_def

,index,BiGG_ID,query,hit,mg_L,mol_weight,mmol/L
0,0,ala_L(e),L-Alanine,L-Alanine,655.85000,89.09322,7.361391
1,1,arg_L(e),L-Arginine,L-Arginine,952.33000,174.20100,5.466846
2,2,asp_L(e),L-Aspartate,L-Aspartic acid,1166.45000,133.10270,8.763534
3,3,cys_L(e),L-Cysteine,L-Cysteine,144.44000,121.15800,1.192162
4,4,glu_L(e),L-Glutamate,L-Glutamic acid,2324.23000,147.12930,15.797193
5,5,gly(e),Glycine,Glycine,1126.31000,75.06664,15.004135
6,6,ile_L(e),L-Isoleucine,L-Isoleucine,710.11000,131.17500,5.413455
7,7,leu_L(e),L-Leucine,L-Leucine,911.12000,131.17296,6.945944
8,8,lys_L(e),L-Lysine,L-Lysine,875.00000,146.18764,5.985458
9,9,met_L(e),L-Methionine,L-Methionine,240.42000,149.21238,1.611260


In [181]:
vitamins=Mol_Weight(["Thiamin", "Riboflavin", "Reduced Riboflavin", "Nicotinate", "Nicotinamide",  "Pyridoxamine", "Pyridoxal","Pyridoxine", "Folate", "Pantothenate", "Biotin"])
vitamins["mol_weight"][2]=vitamins["mol_weight"][1]
vitamins

,query,hit,hmdb_id,kegg_id,pubchem_id,chebi_id,metlin_id,smiles,mol_weight
0,Thiamin,Thiamine,HMDB0000235,C00378,1130,18385,5242,CC1=C(SC=[N+]1CC2=CN=C(N=C2N)C)CCO,265.35574
1,Riboflavin,Riboflavin,HMDB0000244,C00255,493570,17015,5249,CC1=C(C)C=C2N(C[C@H](O)[C@H](O)[C@H](O)CO)C3=N...,376.36390
2,Reduced Riboflavin,Riboflavin reduced,HMDB0001557,C01007,22833571,-,6322,OC[C@@H](O)[C@H](O)[C@H](O)CN1C2=CC=CC=C2N=C2C...,376.36390
3,Nicotinate,Nicotinic acid,HMDB0001488,C00253,938,15940,6272,OC(=O)C1=CN=CC=C1,123.10944
4,Nicotinamide,Niacinamide,HMDB0001406,C00153,936,17154,1497,NC(=O)C1=CC=CN=C1,122.12472
5,Pyridoxamine,Pyridoxamine,HMDB0001431,C00534,1052,16410,238,CC1=NC=C(C(=C1O)CN)CO,168.19316
6,Pyridoxal,Pyridoxal,HMDB0001545,C00250,1050,17310,6312,CC1=NC=C(CO)C(C=O)=C1O,167.16200
7,Pyridoxine,Pyridoxine,HMDB0000239,C00314,1054,16709,5245,CC1=NC=C(C(=C1O)CO)CO,169.17788
8,Folate,Folic acid,HMDB0000121,C00504,6037,27470,246,NC1=NC(=O)C2=NC(CNC3=CC=C(C=C3)C(=O)N[C@@H](CC...,441.39750
9,Pantothenate,Pantothenic acid,HMDB0000210,C00864,6613,46905,-,CC(C)(CO)[C@@H](O)C(=O)NCCC(O)=O,219.23502


In [182]:
#Vitamins

# - thm(e)' thiamin vitamin b1
#    - biotin -> btn |from yeast extract? 
#    - 'rbflvrd(e)', 'ribflv(e)' vitamin b2
#    - 'pydam(e)' pydx pydxn vitamin b6 from yeast extract
#    - 'pnto_R(e)' pantothenic acid from yeast extract?
#    - 'ptrc(e)' probably from meat extract(e)
#    - 'spmd(e)' from meat extract and linked to ptrc
#    - cbl1   Cyanocobalamin is a cobalt-containing coordination compound and a natural water-soluble vitamin of the B-complex family 
#    - adocbl
#    - folate fol is of B vitamin family 
#    - tautin from meat extract
#    - nac is nicotinate and also vitamin b and so is nacm nicotinamide


#vitamins["query"], mol_weight_vitamins, vitamins_mmol_l
#https://www.sigmaaldrich.com/catalog/product/sial/07533?lang=de&region=DE 
#Vitamins mg/100 g product
#Thiamin HCI (B1) ≥36
#Riboflavin (B2) ≥13
#Niacin ≥650
#Vitamin B6 3.9 #pyridoxine
#Folic acid 2.8
#Pantothenic acid 11.2
#Biotin ≥2
#Inositol ≥160
#Pyridoxine HCI 7

Bact_Peptone_L37#g/L
Proteose_Peptone_L85
soya_peptone_L44
yeast_extract
Lab_Lemco_L29
Liver_Digest_Neutralized_L27
mg_100g_to_ppm =1/10000
µg_to_mg=1/1000
nb_extracts=6

vitamins_mg_100g=np.array([36,13,650/2, 650/2,3.9/3, 3.9/3, 3.9/3,2.8,11.2,2,7])
vitamins_ppm=vitamins_mg_100g * mg_100g_to_ppm
vitamins_µg_L=vitamins_ppm*(yeast_extract+Bact_Peptone_L37+Proteose_Peptone_L85+soya_peptone_L44+Lab_Lemco_L29+Liver_Digest_Neutralized_L27)
vitamins_mg_L=vitamins_µg_L*µg_to_mg
vitamins_mmol_l=vitamins_mg_L/vitamins["mol_weight"]

vitamin_biggs=["thm(e)","ribflv(e)", "rbflvrd(e)", "nac(e)","ncam(e)", "pydam(e)", "pydx(e)", "pydxn(e)","fol(e)","pnto_R(e)", "btn(e)"]
pd.DataFrame()

vitamins["mmol/L"]=vitamins_mmol_l
vitamins["BiGG_ID"]=vitamin_biggs
Traces_AA_def_Vit=pd.concat([Traces_AA_def, vitamins], join="inner", ignore_index=True)
Traces_AA_def_Vit

,BiGG_ID,query,hit,mol_weight,mmol/L
0,ala_L(e),L-Alanine,L-Alanine,89.09322,7.361391e+00
1,arg_L(e),L-Arginine,L-Arginine,174.20100,5.466846e+00
2,asp_L(e),L-Aspartate,L-Aspartic acid,133.10270,8.763534e+00
3,cys_L(e),L-Cysteine,L-Cysteine,121.15800,1.192162e+00
4,glu_L(e),L-Glutamate,L-Glutamic acid,147.12930,1.579719e+01
5,gly(e),Glycine,Glycine,75.06664,1.500413e+01
6,ile_L(e),L-Isoleucine,L-Isoleucine,131.17500,5.413455e+00
7,leu_L(e),L-Leucine,L-Leucine,131.17296,6.945944e+00
8,lys_L(e),L-Lysine,L-Lysine,146.18764,5.985458e+00
9,met_L(e),L-Methionine,L-Methionine,149.21238,1.611260e+00


In [183]:
comparison=pd.DataFrame({"Compound":MediumMgam, "Specific_Bound_Set":pd.Series(MediumMgam).isin(Traces_AA_def_Vit["BiGG_ID"])})
no_specific_bound=comparison.loc[comparison["Specific_Bound_Set"]==False]
no_specific_bound["Compound"], len(no_specific_bound), len(MediumMgam)

(0       Lcyst(e)
 1     Lcystin(e)
 2         ade(e)
 3      adocbl(e)
 9        cbl1(e)
 10       cbl2(e)
 12    cobalt2(e)
 21      gthox(e)
 22      gthrd(e)
 23        gua(e)
 25        h2o(e)
 26       hxan(e)
 28        ins(e)
 42       ptrc(e)
 49        so4(e)
 50       spmd(e)
 52       taur(e)
 55      thymd(e)
 58        ura(e)
 60        xan(e)
 61      xyl_D(e)
 Name: Compound, dtype: object,
 21,
 62)

In [185]:
MediumMgam_x=MediumMgam

model_mgam=medium(model.copy(),MediumMgam_x)
print('\nMedium_mgam')
for j,i in enumerate(Traces_AA_def_Vit["BiGG_ID"]):
    model_mgam.exchanges.get_by_id("EX_"+i).lower_bound=-Traces_AA_def_Vit.loc[j,"mmol/L"]
for i in no_specific_bound["Compound"]:
    model_mgam.exchanges.get_by_id("EX_"+i).lower_bound=-0.02
model_mgam.summary()


Medium_mgam


Metabolite,Reaction,Flux,C-Number,C-Flux
ade[e],EX_ade(e),1.919E-06,5,0.00%
ala-L[e],EX_ala_L(e),7.488E-06,3,0.00%
arg-L[e],EX_arg_L(e),3.668E-06,6,0.00%
asp-L[e],EX_asp_L(e),6.303E-06,4,0.00%
glc-D[e],EX_glc(e),2.775,6,96.80%
glu-L[e],EX_glu_L(e),5.822E-06,5,0.00%
gly[e],EX_gly(e),5.512E-06,2,0.00%
gua[e],EX_gua(e),2.24E-06,5,0.00%
ins[e],EX_ins(e),0.02,10,1.16%
leu-L[e],EX_leu_L(e),4.225E-06,6,0.00%


In [44]:
for i in MediumMgam:
    print(model_mgam.reactions.get_by_id(i))
    print(model_mgam.reactions.get_by_id(i).lower_bound)

EX_Lcyst(e): Lcyst[e] <=> 
-0.02
EX_Lcystin(e): Lcystin[e] <=> 
-0.02
EX_ade(e): ade[e] <=> 
-0.02
EX_adocbl(e): adocbl[e] <=> 
-0.02
EX_ala_D(e): ala-D[e] <=> 
-0.02
EX_ala_L(e): ala-L[e] <=> 
-7.361390687192584
EX_arg_L(e): arg-L[e] <=> 
-5.740495175113805
EX_asp_L(e): asp-L[e] <=> 
-8.763533722456419
EX_btn(e): btn[e] <=> 
-0.02
EX_ca2(e): ca2[e] <=> 
-0.14130196117570737
EX_cbl1(e): cbl1[e] <=> 
-0.02
EX_cbl2(e): cbl2[e] <=> 
-0.02
EX_cl(e): cl[e] <=> 
-93.0817681022884
EX_cobalt2(e): cobalt2[e] <=> 
-0.02
EX_cu2(e): cu2[e] <=> 
-0.0013344663708179901
EX_cys_L(e): cys-L[e] <=> 
-2.47610558114198
EX_fe2(e): fe2[e] <=> 
-0.009472647506491183
EX_fe3(e): fe3[e] <=> 
-0.009472647506491183
EX_fol(e): fol[e] <=> 
-0.02
EX_glc(e): glc-D[e] <=> 
-2.7753743036308336
EX_gln_L(e): gln-L[e] <=> 
-0.02
EX_glu_L(e): glu-L[e] <=> 
-15.79719335305748
EX_gly(e): gly[e] <=> 
-15.004134992587916
EX_gthox(e): gthox[e] <=> 
-0.02
EX_gthrd(e): gthrd[e] <=> 
-0.02
EX_gua(e): gua[e] <=> 
-0.02
EX_h(e): h[e

In [26]:
fba = model_mgam.optimize()
fluxes = deleteNull(fba.fluxes)
print('\nActive reaction : %i which represent %f%% of the model' %(len(fluxes),len(fluxes)/len(model.reactions)*100))
print(fba.objective_value)


Active reaction : 382 which represent 37.087379% of the model
0.2373312888272731


In [27]:
from cobra.medium import minimal_medium

max_growth = model.slim_optimize()
minimal_medium(model, 0.005)

EX_Lcystin(e)    0.000251
EX_ade(e)        0.001080
EX_adocbl(e)     0.000028
EX_alagln(e)     0.002650
EX_alahis(e)     0.000463
EX_arg_L(e)      0.001441
EX_btn(e)        0.000028
EX_ca2(e)        0.000028
EX_cl(e)         0.000028
EX_cobalt2(e)    0.000028
EX_cps_fp(e)     0.000028
EX_cu2(e)        0.000028
EX_fe3(e)        0.000028
EX_fe3dcit(e)    0.000028
EX_fol(e)        0.000084
EX_glyasn(e)     0.006684
EX_glyc3p(e)     0.009060
EX_glygln(e)     0.001394
EX_gua(e)        0.001163
EX_k(e)          0.000028
EX_metala(e)     0.000775
EX_mg2(e)        0.000028
EX_nac(e)        0.000056
EX_pect(e)       0.008996
EX_pnto_R(e)     0.000056
EX_pydx(e)       0.000028
EX_ribflv(e)     0.000056
EX_ser_L(e)      0.001048
EX_so4(e)        0.000028
EX_thm(e)        0.000028
EX_trp_L(e)      0.000276
dtype: float64

In [30]:
from cobra.medium import minimal_medium
max_growth = model_mgam.slim_optimize()
minimal_medium(model_mgam, max_growth)

EX_Lcystin(e)    0.011893
EX_ade(e)        0.020000
EX_adocbl(e)     0.001334
EX_ala_D(e)      0.020000
EX_ala_L(e)      0.164522
EX_arg_L(e)      0.122112
EX_btn(e)        0.001334
EX_ca2(e)        0.001334
EX_cl(e)         0.001334
EX_cobalt2(e)    0.001334
EX_cu2(e)        0.001334
EX_fe3(e)        0.002669
EX_fol(e)        0.004003
EX_gln_L(e)      0.020000
EX_glu_L(e)      0.174594
EX_gua(e)        0.020000
EX_his_L(e)      0.020000
EX_hxan(e)       0.020000
EX_ins(e)        0.020000
EX_k(e)          0.001334
EX_leu_L(e)      0.104108
EX_lys_L(e)      0.079162
EX_met_L(e)      0.036778
EX_mg2(e)        0.001334
EX_nac(e)        0.002669
EX_pi(e)         0.440703
EX_pnto_R(e)     0.002669
EX_pydx(e)       0.001334
EX_ribflv(e)     0.002669
EX_ser_L(e)      0.923712
EX_so4(e)        0.001334
EX_strch1(e)     1.180099
EX_thm(e)        0.001334
EX_thymd(e)      0.006912
EX_trp_L(e)      0.013090
EX_ura(e)        0.020000
EX_xan(e)        0.020000
dtype: float64

---
## Reinforced Clostridial Medium (RCM)
http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=CM0149&org=53&c=UK&lang=EN

* Yeast Extract -> [lots of stuff in it, unsure of what's garbage](https://www.chemicalbook.com/ChemicalProductProperty_EN_CB9440339.htm#:~:text=Yeast%20extract%20has%20a%20protein,aromatic%20compounds%20and%20other%20components.)
    * Glutathione: gthrd (reduced version)
    * 18 AA's: see above
    * Dextran: glc__D (it's made of glucose)
    * Mannan: mannan
    * Trehalose: tre
    * B-vitamins: (Clemence mentioned vitamins being important so I'm putting them all in)
        * \1. Thiamin -> thm
        * \2. Riboflavin -> ribflv
        * \3. Niacin -> trp__L (made of tryptophan)
        * \5. Pantothenic Acid -> pnto__R
        * \6. Pyridoxine -> pydxn
        * \7. Biotin -> btn
        * \9. Folic Acid -> ... can't find anything
        * \12. Cobalamin -> b12 (or cbl1)
    * Biotin
* Peptone -> Sticking with the above logic, probably a source of all AAs
* Glucose -> Assuming D_Glucose: glc__D
* Soluble Starch -> starch1200
* Sodium Chloride -> na1, cl
* Sodium Acetate -> Assuming breaking down into sodium and acetate: ac, na1
* Cysteine Hydrochloride -> cys_L, cl, h (could also be cys_D? cys_L is more widely used)
* Agar: Agarose (70%) and Agaropectin (30%) [according to Wiki](https://en.wikipedia.org/wiki/Agar)
    * Agarose: D-galactose & 3,6-anhydro-L-galactopyranose
        * D-galactose: gal
        * 3,6-anhydro-L-galactopyranose: [similar structure & compositon to beta-D-allose](https://pubchem.ncbi.nlm.nih.gov/#query=CID67020466%20structure&tab=similarity)
            * D-Allose: all__D
    * Agaropectin: D-galactose, L-galactose, pyruvate, sulfate
        * D-galactose: gal
        * L-galactose: gal__L
        * Pyruvate: pyr
        * Sulfate: so4

| Component              | Concentration (g/L) | BiGG Metabolites                                                                          |
|------------------------|---------------------|-------------------------------------------------------------------------------------------|
| Yeast Extract          | 13.0                | (see all AA's), gthrd, glc\__D, mannan, tre, thm, ribflv, trp\__L, pnto\__R, pydxn, btn, b12 |
| Peptone                | 10.0                | (see all AA's)                                                                            |
| Glucose                | 5.0                 | glc_D                                                                                     |
| Soluble Starch         | 1.0                 | starch1200                                                                                |
| Sodium Chloride        | 5.0                 | na1, cl                                                                                   |
| Sodium Acetate         | 3.0                 | na1, ac                                                                                   |
| Cysteine Hydrochloride | 0.5                 | cys_L, cl, h                                                                              |
| Agar                   | 0.5                 | gal, all\__D, gal\__L, pyr, so4                                                           |

In [ ]:
medium_rcm = list(('gthrd','glc__D','mannan','tre','thm','ribflv','trp__L','pnto__R','pydxn','btn','b12',
             'starch1200','na1','ac','cl','cys_L','gal','all__D','gal__L','pyr','so4'))
medium_rcm=[mets+"(e)" for mets in medium_rcm]
model_rcm=medium(model.copy(), medium_rcm)
for i in range(1,len(model_rcm.reactions)):
    model_rcm.reactions[i].upper_bound=1000
    model_rcm.reactions[i].lower_bound=-1000
model_rcm.objective = {model_rcm.reactions.get_by_id('Biomass_FP'): 1}
model_rcm.summary()

In [ ]:
model_mmcb.reactions.get_by_id('mannan')

In [ ]:
# Post-optimization
fba_rcm = model_rcm.optimize()
fluxes = deleteNull(fba_rcm.fluxes)
print('\nActive reaction : %i which represent %f%% of the model' %(len(fluxes),len(fluxes)/len(model.reactions)*100))
print(model_rcm.objective.value)

---
## mMCB

(There are also supplements mentioned in the paper but this is the base medium)
* [Bacteriological Peptome (6.5) (Oxoid)](http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=LP0037&c=UK&lang=EN): 
    * Vague, decided to go with polypeptides: <b>polypep</b>
    * Nitrogen: <b>n2</b>
* [Soy Peptome (5.0) (Oxoid)](http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=LP0044&cat=&c=UK&lang=EN) --> another vague one:
    * Stachyose: <b>stys</b>
    * Raffinose: <b>raffin</b>
    * Sucrose: <b>sucr</b>
    * Nitrogen: <b>n2</b>
* [Yeast Extract (3.0) (VWR International, Darmstadt, Germany)](https://us.vwr.com/store/product/7437401/vwr-life-science-yeast-extract-bacteriological-grade):
    * Vitamin B (same logic as RCM):
        * 1. Thiamin -> <b>thm</b>
        * 2. Riboflavin -> <b>ribflv</b>
        * 3. Niacin -> <b>trp__L</b> (made of tryptophan)
        * 5. Pantothenic Acid -> <b>pnto__R</b>
        * 6. Pyridoxine -> <b>pydxn</b>
        * 7. Biotin -> <b>btn</b>
        * 9. Folic Acid -> ... can't find anything
        * 12. Cobalamin -> <b>b12</b> (or cbl1)
* [Tryptone (2.5) (Oxoid)](http://www.oxoid.com/UK/blue/prod_detail/prod_detail.asp?pr=LP0042&c=UK&lang=EN): 
    * Tryptophan: <b>trp__L</b>
* NaCL (1.5) (VWR International, Darmstadt, Germany):
    * NaCl: <b>na1, cl</b>
* K<sub>2</sub>HPO<sub>4</sub> (1.0) (Merck International, Darmstadt, Germany):
    * Potassium: <b>k</b>
    * Phosphate: <b>p1</b>
    * Hydrogen: <b>h2</b>
* KH<sub>2</sub>PO<sub>4</sub> (1.0) (Merck International, Darmstadt, Germany):
    * Potassium: <b>k</b>
    * Phosphate: <b>p1</b>
    * Hydrogen: <b>h2</b>
* Na<sub>2</sub>SO<sub>4</sub> (2.0) (VWR):
    * Sodium: <b>na1</b>
    * Sulfate: <b>so4</b>
* MgSO<sub>4</sub>*7H<sub>2</sub>O (1.0) (Merck):
    * Magnesium: <b>mg2</b>
    * Sulfate: <b>so4</b>
* CaCl<sub>2</sub>*2H<sub>2</sub>O (0.1) (Merck):
    * Calcium Chloride: <b>ca2, cl</b>
* NH<sub>4</sub>Cl (1.0) (Merck):
    * Ammonium Chloride: <b>nh4, cl</b>
* Cysteine-HCL (0.4) (Merck):
    * Cysteine: <b>cys__L</b>
    * HCL: <b>h2, cl</b>
* NaHCO<sub>3</sub> (0.2) (VWR):
    * Sodium: <b>na1</b>
    * Bicarbonate: <b>hco3<b>
* MnSO<sub>4</sub>*H<sub>2</sub>O (0.05) (VWR):
    * Manganese: <b>mn2</b>
    * Sulfate: <b>so4</b>
* FeSO<sub>4</sub>*7H<sub>2</sub>O (0.005) (Merck):
    * Iron: <b>fe</b>
    * Sulfate: <b>so4</b>
* ZnSO<sub>4</sub>*7H<sub>2</sub>O (0.005) (VWR):
    * Zinc: <b>zn2</b>
    * Sulfate: <b>so4</b>
* Hemin (0.005) (Sigma-Aldrich, Steinheim, Germany):
    * ... contains iron
* Menadione (0.005) (S-A):
    * <b>mndn</b>
* Resazurin (0.001) (S-A):
    * Fluoro identifier



| Component                            | Concentration (g/L) | BiGG Metabolites                                |
|--------------------------------------|---------------------|-------------------------------------------------|
| Bacteriological Peptome              | 6.5                 | polypep, n2                                     |
| Soy Peptome                          | 5.0                 | stys, raffin, sucr, n2                          |
| Yeast Extract                        | 3.0                 | thm, ribflv, trp\__L, pnto\__R, pydxn, btn, b12 |
| Tryptone                             | 2.5                 | trp\__L                                         |
| NaCL                                 | 1.5                 | na1, cl                                         |
| K<sub>2</sub>PO<sub>4</sub>          | 1.0                 | k, p1, h2                                       |
| KH<sub>2</sub>PO<sub>4</sub>         | 1.0                 | k, p1, h2                                       |
| Na<sub>2</sub>SO<sub>4</sub>         | 2.0                 | na1, so4                                        |
| Mg<sub>2</sub>SO<sub>4</sub>         | 1.0                 | mg2, so4                                        |
| CaCl<sub>2</sub>*2H<sub>2</sub>O     | 0.1                 | ca2, cl                                         |
| NH<sub>4</sub>Cl                     | 1.0                 | nh4, cl                                         |
| Cysteine-HCL                         | 0.4                 | cys__L, h2, cl                                  |
| NaHCO<sub>3</sub>                    | 0.2                 | na1, hco3                                       |
| MnSO<sub>4</sub>*H<sub>2</sub>O      | 0.05                | mn2, so4                                        |
| FeSOMnSO<sub>4</sub>*7H<sub>2</sub>O | 0.005               | fe, so4                                         |
| ZnSO<sub>4</sub>*7H<sub>2</sub>O     | 0.005               | zn2, so4                                        |
| Hemin                                | 0.005               |                                                 |
| Menadione                            | 0.005               | mndn                                            |
| Resazurin                            | 0.001               |                                                 |

In [ ]:
model.copy()

In [ ]:
medium_mmcb = list(('polypep','n2','stys','raffin','sucr','n2','thm','ribflv','trp__L',
                    'pnto__R','pydxn','btn','b12','trp__L','na1','cl','k','p1','h2','na1',
                    'so4','mg2','ca2','nh4','cys__L','h2','cl','na1','hco3','mn2','zn2','mndnb'))

In [ ]:
medium_mmcb=[mets+"(e)" for mets in medium_mmcb]
model_mmcb=medium(model.copy(), medium_mmcb)
unused = []
for i in range(1,len(model_mmcb.reactions)):
    model_mmcb.reactions[i].upper_bound=1000
    if model_mmcb.reactions[i].lower_bound != 0:
        model_mmcb.reactions[i].lower_bound=-10
    else:    
        model_mmcb.reactions[i].lower_bound=-
model_mmcb.objective = {model_mmcb.reactions.get_by_id('Biomass_FP'): 1}
model_mmcb.summary()

# Find list of all metabolites not in media
# Express in mmol/g/L
# Set as -'ve flux lower bound

In [ ]:
# Post-optimization
fba_mmcb = model_mmcb.optimize()
fluxes_mmcb = deleteNull(fba_mmcb.fluxes)
print('\nActive reaction : %i which represent %f%% of the model' %(len(fluxes_mmcb),len(fluxes_mmcb)/len(model.reactions)*100))
print(model_mmcb.objective.value)